In [35]:
import pandas as pd
import gzip
%matplotlib inline
import numpy as np
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
df_reviews = getDF('reviews_Home_and_Kitchen_5.json.gz')

In [4]:
df_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,APYOBQE6M18AA,0615391206,Martin Schwartz,"[0, 0]",My daughter wanted this book and the price on ...,5.0,Best Price,1382140800,"10 19, 2013"
1,A1JVQTAGHYOL7F,0615391206,Michelle Dinh,"[0, 0]",I bought this zoku quick pop for my daughterr ...,5.0,zoku,1403049600,"06 18, 2014"
2,A3UPYGJKZ0XTU4,0615391206,mirasreviews,"[26, 27]",There is no shortage of pop recipes available ...,4.0,"Excels at Sweet Dessert Pops, but Falls Short ...",1367712000,"05 5, 2013"
3,A2MHCTX43MIMDZ,0615391206,"M. Johnson ""Tea Lover""","[14, 18]",This book is a must have if you get a Zoku (wh...,5.0,Creative Combos,1312416000,"08 4, 2011"
4,AHAI85T5C2DH3,0615391206,PugLover,"[0, 0]",This cookbook is great. I have really enjoyed...,4.0,A must own if you own the Zoku maker...,1402099200,"06 7, 2014"


In [5]:
df_reviews.shape

(551682, 9)

In [6]:
df_reviews['bin_rating']=0

In [7]:
for i in range(20000):
    if df_reviews.loc[i,'overall']<5.0:
        df_reviews.loc[i,'bin_rating']=0
    else: 
        df_reviews.loc[i,'bin_rating']=1

In [8]:
df_short=df_reviews.iloc[:20000,:].copy()

In [9]:
keywords = ['best', 'happy', 'quick', 'love', 'beautiful', 'worth','inspire','recommend','highly',
            'hoping','better','hard','thought','busted','fell off','quality','low','prefer','broken','terrible'
           'good', 'great', 'ideal', 'nice', 'excellent', 'highly','loved',
            'fine','sweetest','excellent','wow','satisfied','cool',
            'comfortable','perfect','simple','strong','easy',
            'darn','return','refund','wrong','joke','failed','dead','uncomfortable',
            'junk','never','blew up','unsatisfactory','waste','fails',
            'poor','worthless','disappointed','trouble','not']

for key in keywords:
    df_short[str(key)] = df_short.reviewText.str.contains(str(key), case=False)

In [10]:
df_train=df_short.iloc[0:9999,:].copy()

In [11]:
df_test=df_short.iloc[10000:19999,:].copy()

In [12]:
training_features = df_train[keywords]
training_target = df_train['bin_rating']
test_features = df_test[keywords]
test_target = df_test['bin_rating']
bnb = BernoulliNB()
bnb.fit(training_features, training_target)

train_pred = bnb.predict(training_features)
print("Number of mislabeled points out of a total {} training points : {}".format(
    df_train.shape[0],
    (training_target != train_pred).sum()
))


test_pred = bnb.predict(test_features)
print("Number of mislabeled points out of a total {} test points : {}".format(
    df_test.shape[0],
    (test_target != test_pred).sum()
))

Number of mislabeled points out of a total 9999 training points : 2754
Number of mislabeled points out of a total 9999 test points : 2932


In [13]:
df_train.bin_rating.sum()

6904

In [14]:
df_test.bin_rating.sum()

6652

In [19]:
# Random Forest Approach
rfc = ensemble.RandomForestClassifier()
training_features = df_train[keywords]
training_target = df_train['bin_rating']

cross_val_score(rfc, training_features, training_target, cv=5)

array([ 0.676     ,  0.6885    ,  0.6925    ,  0.6785    ,  0.70085043])

In [21]:
# Gradient Boosting Approach
training_features = df_train[keywords]
training_target = df_train['bin_rating']
test_features = df_test[keywords]
test_target = df_test['bin_rating']
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(training_features, training_target)

predict_train = clf.predict(training_features)
predict_test = clf.predict(test_features)

# Accuracy tables.
table_train = pd.crosstab(training_target, predict_train, margins=True)
table_test = pd.crosstab(test_target, predict_test, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(train_tI_errors, train_tII_errors, test_tI_errors, test_tII_errors))

Training set accuracy:
Percent Type I errors: 0.22162216221622164
Percent Type II errors: 0.043804380438043806

Test set accuracy:
Percent Type I errors: 0.23662366236623664
Percent Type II errors: 0.058105810581058105


In [34]:
# try Logistic Regression

# Declare a logistic regression classifier, with small parameter regularization coefficient C.
lr = LogisticRegression(C=1e9)

training_features = df_train[keywords]
training_target = df_train['bin_rating']
test_features = df_test[keywords]
test_target = df_test['bin_rating']

# Fit the model.
fit = lr.fit(training_features, training_target)

print(lr.score(test_features,test_target))

0.705070507051


In [36]:
# try a Support Vector classifier
svc = SVC()
training_features = df_train[keywords]
training_target = df_train['bin_rating']
test_features = df_test[keywords]
test_target = df_test['bin_rating']
svc.fit(training_features,training_target)
svc.score(test_features,test_target)

0.67626762676267627